In [ ]:
!git clone https://github.com/facebookresearch/sam2.git
!pip install -q matplotlib opencv-python
%cd sam2 
!pip install -e .

!mkdir -p checkpoints
!wget -P checkpoints https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_small.pt

Cloning into 'sam2'...
remote: Enumerating objects: 1070, done.
remote: Total 1070 (delta 0), reused 0 (delta 0), pack-reused 1070 (from 1)
Receiving objects: 100% (1070/1070), 128.11 MiB | 40.46 MiB/s, done.
Resolving deltas: 100% (380/380), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 106.7 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requi

In [2]:
%cd ..

/kaggle/working


In [3]:
!pip install -q --upgrade dataset-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 28.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 113.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 109.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install -q peft bitsandbytes

In [ ]:
import sys
import os

if os.path.exists('/kaggle/working/sam2'):
    sys.path.append('/kaggle/working/sam2')
    print("Path sam2 berhasil ditambahkan!")
else:
    print("Folder sam2 tidak ditemukan, pastikan lokasi clone benar.")

Path sam2 berhasil ditambahkan!


In [ ]:
import os
import sys

print(f"Direktori awal: {os.getcwd()}")

if os.path.exists("/kaggle/working/sam2") and os.getcwd() != "/kaggle/working/sam2":
    os.chdir("/kaggle/working/sam2")
    print(f"Berhasil pindah ke: {os.getcwd()}")

if "/kaggle/working/sam2" not in sys.path:
    sys.path.append("/kaggle/working/sam2")



Direktori awal: /kaggle/working
Berhasil pindah ke: /kaggle/working/sam2


In [8]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import shutil
from torch.utils.data import Dataset, DataLoader
from IPython.display import FileLink, display
from peft import get_peft_model, LoraConfig, TaskType
import bitsandbytes as bnb

IMAGE_DIR = "/kaggle/input/high-resolution-viton-zalando-dataset/train/image"
MASK_DIR  = "/kaggle/input/high-resolution-viton-zalando-dataset/train/image-parse-v3" 

CHECKPOINT = "/kaggle/working/sam2/checkpoints/sam2_hiera_small.pt"
CONFIG     = "sam2_hiera_s.yaml"
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"

DATA_SIZE = 1024       
BATCH_SIZE = 2         
LR = 1e-4              
EPOCHS = 5             
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

FINAL_FILENAME = "sam2_finetuned.pth"
OUTPUT_DIR = "/kaggle/working/output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

from sam2.build_sam import build_sam2

class VITONDataset(Dataset):
    def __init__(self, image_dir, mask_dir, size=1024):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.size = size
        self.images = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg','.jpeg','.png'))]
        if len(self.images) == 0:
            raise RuntimeError(f"No images found in {image_dir}")
        print(f"[DATA] {len(self.images)} VITON images found.")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)
        base = os.path.splitext(img_name)[0]
        
        mask_path = os.path.join(self.mask_dir, base + ".png")
        if not os.path.exists(mask_path):
            mask_path = os.path.join(self.mask_dir, base + ".jpg")
            if not os.path.exists(mask_path):
                 mask_path = os.path.join(self.mask_dir, base + "_label.png")

        image = cv2.imread(img_path)
        mask = None
        if os.path.exists(mask_path):
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if image is None or mask is None:
            new_idx = (idx + 1) % len(self)
            return self.__getitem__(new_idx)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (self.size, self.size))
        mask = cv2.resize(mask, (self.size, self.size), interpolation=cv2.INTER_NEAREST)
        binary_mask = (mask > 0).astype(np.float32)

        if binary_mask.sum() == 0:
            new_idx = (idx + 1) % len(self)
            return self.__getitem__(new_idx)

        ys, xs = np.where(binary_mask > 0)
        x_min, x_max = xs.min(), xs.max()
        y_min, y_max = ys.min(), ys.max()

        noise = np.random.randint(-10, 10, size=4)
        box = np.array([x_min, y_min, x_max, y_max]) + noise
        box = np.clip(box, 0, self.size).astype(np.float32)

        image = image.astype(np.float32) / 255.0
        image = torch.tensor(image).permute(2,0,1).contiguous()
        binary_mask = torch.tensor(binary_mask).unsqueeze(0).float()
        box = torch.tensor(box).float()

        return image, binary_mask, box

print("[MODEL] Building SAM2 model...")
model = build_sam2(CONFIG, CHECKPOINT, device=DEVICE)

# 1. Identifikasi layer Linear di dalam Mask Decoder untuk dipasang LoRA
target_modules = []
for name, module in model.sam_mask_decoder.named_modules():
    if isinstance(module, torch.nn.Linear):
        full_name = f"sam_mask_decoder.{name}"
        target_modules.append(full_name)

print(f"[PEFT] Found {len(target_modules)} linear layers in Mask Decoder to target.")

# 2. Konfigurasi LoRA menggunakan PEFT
peft_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=target_modules,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type=None, 
)

# 3. Wrap model dengan PEFT
model = get_peft_model(model, peft_config)
model.to(DEVICE)

# 4. Unfreeze Prompt Encoder
for n, p in model.base_model.model.sam_prompt_encoder.named_parameters():
    p.requires_grad = True

model.print_trainable_parameters()

dataset = VITONDataset(IMAGE_DIR, MASK_DIR, size=DATA_SIZE)
print("[DATA] Dataset length:", len(dataset))
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)

print("[OPTIMIZER] Using bitsandbytes 8-bit AdamW")
optimizer = bnb.optim.AdamW8bit(filter(lambda p: p.requires_grad, model.parameters()), lr=LR)

loss_fn = nn.BCEWithLogitsLoss()
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE == "cuda"))

model.train()
print("\n--- MULAI TRAINING VITON (PEFT + BNB 8-bit) ---")

try:
    for epoch in range(EPOCHS):
        print(f"\n[Epoch {epoch+1}/{EPOCHS}]")
        total_loss = 0.0
        steps = 0

        for i, (images, gt_masks, boxes) in enumerate(dataloader):
            images = images.to(DEVICE, non_blocking=True)
            gt_masks = gt_masks.to(DEVICE, non_blocking=True)
            boxes = boxes.to(DEVICE, non_blocking=True)

            optimizer.zero_grad()

            with torch.no_grad():
                if DEVICE == "cuda":
                    with torch.cuda.amp.autocast():
                        backbone_out = model.forward_image(images)
                else:
                    backbone_out = model.forward_image(images)

            boxes_for_encoder = boxes.unsqueeze(1)
            sparse_embeddings, dense_embeddings = model.base_model.model.sam_prompt_encoder(
                points=None,
                boxes=boxes_for_encoder,
                masks=None
            )

            image_embed = backbone_out.get("vision_features", None)
            image_pe = model.base_model.model.sam_prompt_encoder.get_dense_pe()
            
            if "backbone_fpn" in backbone_out:
                fpn = backbone_out["backbone_fpn"]
                if isinstance(fpn, (list, tuple)) and len(fpn) >= 2:
                    high_res_features = [fpn[0], fpn[1]]
                else:
                    high_res_features = list(fpn) if isinstance(fpn, (list, tuple)) else [fpn]
            else:
                high_res_features = None

            with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
                decoder_out = model.base_model.model.sam_mask_decoder(
                    image_embeddings=image_embed,
                    image_pe=image_pe,
                    sparse_prompt_embeddings=sparse_embeddings,
                    dense_prompt_embeddings=dense_embeddings,
                    multimask_output=False,
                    repeat_image=False,
                    high_res_features=high_res_features,
                )

                if isinstance(decoder_out, tuple) or isinstance(decoder_out, list):
                     low_res_masks = decoder_out[0]
                else:
                     low_res_masks = decoder_out

                pred_masks = torch.nn.functional.interpolate(
                    low_res_masks,
                    size=(gt_masks.shape[2], gt_masks.shape[3]),
                    mode="bilinear",
                    align_corners=False
                )
                
                loss = loss_fn(pred_masks, gt_masks)

            if DEVICE == "cuda":
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()

            total_loss += loss.item()
            steps += 1

            if i % 10 == 0:
                print(f"[Epoch {epoch+1}] Step {i} | BCE Loss: {loss.item():.6f}")

        avg_loss = total_loss / steps if steps > 0 else 0.0
        print(f"\n=== Epoch {epoch+1} Selesai | Avg BCE Loss: {avg_loss:.6f} ===")
        print("[SAVE] Merging weights and saving single .pth file...")
        
        save_path = os.path.join(OUTPUT_DIR, FINAL_FILENAME)
        
        torch.save(model.state_dict(), save_path)
        print(f"[SAVE] Full PEFT checkpoint saved to: {save_path}")

    print("\n" + "="*40)
    print(f"TRAINING SELESAI. File tersimpan di: {save_path}")
    print("="*40)
    
    relative_path = os.path.join("output", FINAL_FILENAME)
    if os.path.exists(save_path):
        display(FileLink(relative_path))
    else:
        print("File tidak ditemukan.")

except Exception as e:
    print("[ERROR] Crash occurred:")
    import traceback
    traceback.print_exc()

2025-12-18 14:24:06.393019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766067846.571060      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766067846.619271      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

[MODEL] Building SAM2 model...
[PEFT] Found 50 linear layers in Mask Decoder to target.
trainable params: 236,660 || all params: 46,274,282 || trainable%: 0.5114
[DATA] 11647 VITON images found.
[DATA] Dataset length: 11647
[OPTIMIZER] Using bitsandbytes 8-bit AdamW

--- MULAI TRAINING VITON (PEFT + BNB 8-bit) ---

[Epoch 1/5]


/tmp/ipykernel_47/2613205530.py:130: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE == "cuda"))
/tmp/ipykernel_47/2613205530.py:150: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/tmp/ipykernel_47/2613205530.py:174: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):


[Epoch 1] Step 0 | BCE Loss: 0.267712
[Epoch 1] Step 10 | BCE Loss: 0.041734
[Epoch 1] Step 20 | BCE Loss: 0.047942
[Epoch 1] Step 30 | BCE Loss: 0.066249
[Epoch 1] Step 40 | BCE Loss: 0.031446
[Epoch 1] Step 50 | BCE Loss: 0.026553
[Epoch 1] Step 60 | BCE Loss: 0.031714
[Epoch 1] Step 70 | BCE Loss: 0.025866
[Epoch 1] Step 80 | BCE Loss: 0.028618
[Epoch 1] Step 90 | BCE Loss: 0.031356
[Epoch 1] Step 100 | BCE Loss: 0.020659
[Epoch 1] Step 110 | BCE Loss: 0.022472
[Epoch 1] Step 120 | BCE Loss: 0.020400
[Epoch 1] Step 130 | BCE Loss: 0.022779
[Epoch 1] Step 140 | BCE Loss: 0.021875
[Epoch 1] Step 150 | BCE Loss: 0.029401
[Epoch 1] Step 160 | BCE Loss: 0.017893
[Epoch 1] Step 170 | BCE Loss: 0.018344
[Epoch 1] Step 180 | BCE Loss: 0.023489
[Epoch 1] Step 190 | BCE Loss: 0.022257
[Epoch 1] Step 200 | BCE Loss: 0.025559
[Epoch 1] Step 210 | BCE Loss: 0.012296
[Epoch 1] Step 220 | BCE Loss: 0.026193
[Epoch 1] Step 230 | BCE Loss: 0.016594
[Epoch 1] Step 240 | BCE Loss: 0.022145
[Epoch 1] S

/kaggle/working/sam2/output/sam2_finetuned.pth